# Proyecto C - Caso 2

Estudiantes: 

1. Mariana Lozano Roncancio 202122878

# A. Presentaccion de modelo 

A continuacion se presenta el modlo con las siguientes cambios realizados:

- Adición de restricciones de continuidad y consumo de combustible.
- Inclusión de variables auxiliares para orden y combustible restante.
- Restricciones MTZ para evitar subtours.
- Inclusión de la restricción de visita única a cada cliente.
- Reorganización del modelo para mejorar su claridad y consistencia.

## 1. Conjuntos

- **P**: Conjunto de puntos de acceso (puertos)
- **D**: Conjunto de destinos (centros de consumo)
- **E**: Conjunto de estaciones de recarga
- **V**: Conjunto de vehículos
- **N**: Conjunto total de nodos, `N = P ∪ D ∪ E`
- **A**: Conjunto de arcos posibles, `A = {(i, j) ∈ N × N | i ≠ j}`

# 2. Parámetros
- `d_ij`: Distancia entre nodos i y j
- `C_ij`: Costo base entre nodos i y j
- `M_v`: Capacidad útil del vehículo v
- `A_v`: Autonomía del vehículo v
- `R_v`: Rendimiento (km/gal) del vehículo v
- `L_j`: Límite de peso en el nodo j
- `P_s`: Precio de galón en estación s
- `F_t`: Tarifa de flete (COP/km)
- `C_m`: Costo de mantenimiento (COP/km)
- `E_v`: Penalización por CO₂ por km para el vehículo v
- `T_ij`: Peaje base en tramo i → j
- `T2_ij`: Peaje adicional por tonelada
- `demand_j`: Demanda en el nodo j


## 3. Variables de decisión
- `x_ijv ∈ {0,1}`: 1 si el vehículo v viaja de i a j
- `q_ijv ≥ 0`: Carga transportada entre i y j por v
- `z_ijv ≥ 0`: Variable auxiliar para `q * x`
- `f_ijv ≥ 0`: Combustible consumido entre i y j
- `r_sv ≥ 0`: Galones recargados en la estación s por v
- `y_sv ∈ {0,1}`: 1 si v recarga en estación s
- `u_iv ≥ 0`: Orden de visita del nodo i por v
- `comb_iv ≥ 0`: Combustible restante en i por v

## 4. Función Objetivo

Minimizar el costo total de operación:

$$
\min \sum_v \sum_{(i,j)} \left( 
C_{ij} \cdot x_{ijv} + 
F_t \cdot d_{ij} \cdot x_{ijv} + 
C_m \cdot d_{ij} \cdot x_{ijv} + 
T_{ij} \cdot x_{ijv} + 
T2_{ij} \cdot z_{ijv} + 
E_v \cdot d_{ij} \cdot x_{ijv} 
\right) + \sum_v \sum_s P_s \cdot r_{sv}
$$


##  Restricciones

### 1. Conservación de flujo

$$
\sum_{j \in N, j \ne n} x_{jnv} = \sum_{j \in N, j \ne n} x_{njv}
\quad \forall n \in N, \forall v \in V
$$



### 2. Capacidad del vehículo

$$
q_{ijv} \le M_v \cdot x_{ijv}
\quad \forall (i,j) \in A, \forall v \in V
$$



### 3. Límite de peso por municipio

$$
q_{ijv} \le L_j
\quad \forall (i,j) \in A, j \in D, \forall v \in V
$$



### 4. Consumo de combustible

$$
f_{ijv} = \frac{d_{ij}}{R_v} \cdot x_{ijv}
\quad \forall (i,j) \in A, \forall v \in V
$$



### 5. Continuidad del combustible

$$
comb_{jv} = comb_{iv} - f_{ijv} + r_{jv}
\quad \forall (i,j) \in A, \forall v \in V
$$



### 6. Recarga en estación activa

$$
r_{sv} \le M \cdot y_{sv}
\quad \forall s \in E, \forall v \in V
$$



### 7. Eliminación de subtours (MTZ)

$$
u_{iv} - u_{jv} + |D| \cdot x_{ijv} \le |D| - 1
\quad \forall i \ne j \in D, \forall v \in V
$$



### 8. Satisfacción de la demanda

$$
\sum_{i \in N, i \ne j} \sum_{v \in V} q_{ijv} \ge demand_j
\quad \forall j \in D
$$



### 9. Linealización de \( z = q \cdot x \)

$$
\begin{aligned}
z_{ijv} &\le q_{ijv} \\
z_{ijv} &\le M_v \cdot x_{ijv} \\
z_{ijv} &\ge q_{ijv} - M_v \cdot (1 - x_{ijv})
\end{aligned}
$$

# B. Presentacion de Notebook ejecutado

In [1]:
!apt-get install -y coinor-cbc
!pip install pyomo


'apt-get' is not recognized as an internal or external command,
operable program or batch file.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\MarianaLozano\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
import pyomo.environ as pyo
from pyomo.environ import *

# === Cargar datos ===
clients = pd.read_csv("Datos/clients2.csv")
depots = pd.read_csv("Datos/depots2.csv")
stations = pd.read_csv("Datos/stations2.csv")
vehicles = pd.read_csv("Datos/vehicles2.csv")

# === Conjuntos ===
V = list(vehicles['VehicleID'])
D = list(clients['LocationID'])    # Municipios
P = list(depots['DepotID'])        # Puerto
S = list(stations['EstationID'])   # Estaciones de recarga
N = P + D + S                      # Todos los nodos

# === Parámetros ===
# === Parámetros ===
Q = dict(zip(vehicles['VehicleID'], vehicles['Capacity']))             # Capacidad de carga (kg)
Range = dict(zip(vehicles['VehicleID'], vehicles['Range']))            # Autonomía máxima (km)
Demand = dict(zip(clients['LocationID'], clients['Demand']))           # Demanda de cada municipio
FuelPrice = dict(zip(stations['EstationID'], stations['FuelCost']))   # Precio por estación


# === Modelo ===
model = pyo.ConcreteModel()

# Conjuntos
model.V = Set(initialize=V)
model.N = Set(initialize=N)
model.D = Set(initialize=D)
model.S = Set(initialize=S)
model.A = Set(initialize=[(i, j) for i in N for j in N if i != j])

# Variables
model.x = Var(model.V, model.A, within=Binary)  # Ruta
model.u = Var(model.V, model.D, within=NonNegativeReals)  # Subtour elimination (Miller–Tucker–Zemlin)
model.r = Var(model.V, model.S, within=NonNegativeReals)  # Recarga

# Parámetro distancia (ejemplo: reemplazar por tu matriz real)
def distance(i, j):
    # Placeholder: usar Haversine o matriz real
    return np.random.uniform(10, 100)

# === Función Objetivo: Minimizar costo total ===
def total_cost_rule(m):
    route_cost = sum(
        m.x[v, i, j] * distance(i, j) * 5.0  # Costo por distancia recorrida (ejemplo)
        for v in m.V for (i, j) in m.A
    )
    fuel_cost = sum(
        m.r[v, s] * FuelPrice[s]
        for v in m.V for s in m.S
    )
    return route_cost + fuel_cost

model.TotalCost = Objective(rule=total_cost_rule, sense=minimize)

# === Restricciones ===

# 1. Satisfacer la demanda de cada cliente exactamente una vez
def demand_rule(m, d):
    return sum(m.x[v, i, d] for v in m.V for i in m.N if (i, d) in m.A) == 1
model.Demand = Constraint(model.D, rule=demand_rule)

# 2. Restricción de capacidad de carga
def capacity_rule(m, v):
    return sum(Demand[d] * sum(m.x[v, i, d] for i in m.N if (i, d) in m.A) for d in m.D) <= Q[v]
model.Capacity = Constraint(model.V, rule=capacity_rule)

# 3. Flujo de entrada = salida para cada nodo visitado (excepto depot)
def flow_conservation_rule(m, v, n):
    if n in P:
        return Constraint.Skip
    return sum(m.x[v, i, n] for i in m.N if (i, n) in m.A) == sum(m.x[v, n, j] for j in m.N if (n, j) in m.A)
model.Flow = Constraint(model.V, model.N, rule=flow_conservation_rule)

# 4. Subtour elimination (Miller–Tucker–Zemlin)
def subtour_elimination_rule(m, v, i, j):
    if i in P or j in P or i == j:
        return Constraint.Skip
    return m.u[v, i] - m.u[v, j] + len(D) * m.x[v, i, j] <= len(D) - 1
model.Subtour = Constraint(model.V, model.D, model.D, rule=subtour_elimination_rule)

# 5. Restricción de autonomía (combustible disponible >= consumo requerido)
def fuel_constraint_rule(m, v, i, j):
    if (i, j) not in m.A:
        return Constraint.Skip
    return distance(i, j) * 1.0 * m.x[v, i, j] <= Range[v] + sum(
        m.r[v, s] for s in m.S if s == i
    )
model.FuelConstraint = Constraint(model.V, model.N, model.N, rule=fuel_constraint_rule)


# === Resolver ===
solver = SolverFactory('glpk', executable='glpk-4.65\\w64\\glpsol.exe')
solver.options['tmlim'] = 600  # 10 minutos = 600 segundos
results = solver.solve(model, tee=True)


# === Resultados ===
for v in model.V:
    print(f"\nVehículo {v}:")
    route = []
    for (i, j) in model.A:
        if pyo.value(model.x[v, i, j]) > 0.5:
            route.append((i, j))
    print("Ruta:", route)
    recargas = {s: pyo.value(model.r[v, s]) for s in model.S if pyo.value(model.r[v, s]) > 0}
    print("Recargas:", recargas)


GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --tmlim 600 --write C:\Users\MARIAN~1\AppData\Local\Temp\tmp7oz1n9gb.glpk.raw
 --wglp C:\Users\MARIAN~1\AppData\Local\Temp\tmpvqryf_ko.glpk.glp --cpxlp
 C:\Users\MARIAN~1\AppData\Local\Temp\tmp93okvk4f.pyomo.lp
Reading problem data from 'C:\Users\MARIAN~1\AppData\Local\Temp\tmp93okvk4f.pyomo.lp'...
C:\Users\MARIAN~1\AppData\Local\Temp\tmp93okvk4f.pyomo.lp:16987: warning: lower bound of variable 'x2' redefined
C:\Users\MARIAN~1\AppData\Local\Temp\tmp93okvk4f.pyomo.lp:16987: warning: upper bound of variable 'x2' redefined
2049 rows, 1180 columns, 8540 non-zeros
1050 integer variables, all of which are binary
18037 lines were read
Writing problem data to 'C:\Users\MARIAN~1\AppData\Local\Temp\tmpvqryf_ko.glpk.glp'...
14991 lines were written
GLPK Integer Optimizer, v4.65
2049 rows, 1180 columns, 8540 non-zeros
1050 integer variables, all of which are binary
Preprocessing...
999 rows, 1120 columns, 6650 non-zeros

In [ ]:
# === Crear archivo de verificación ===

verif_rows = []

for v in model.V:
    # 1️⃣ Ruta reconstruida como cadena tipo PTO-MUNXX-ESTXX-...-PTO
    route_sequence = []
    current_node = 'PTO'  # Suponiendo que siempre empieza en el puerto (ajusta si tu código tiene otro ID)
    route_sequence.append(current_node)
    
    visited = set()
    while True:
        next_nodes = [j for (i, j) in model.A if i == current_node and pyo.value(model.x[v, i, j]) > 0.5 and j not in visited]
        if not next_nodes:
            break
        next_node = next_nodes[0]
        route_sequence.append(next_node)
        visited.add(next_node)
        current_node = next_node
        if current_node == 'PTO':
            break

    route_string = '-'.join(route_sequence)

    # 2️⃣ Contar municipios visitados
    municipalities_visited = [n for n in route_sequence if n in D]
    municipalities_count = len(municipalities_visited)

    # 3️⃣ Demandas satisfechas (por orden de aparición en la ruta)
    demands_satisfied = []
    for n in route_sequence:
        if n in D:
            demands_satisfied.append(str(Demand[n]))
        elif n in S:
            demands_satisfied.append('0')  # Para estaciones podemos marcar como 0
    demand_string = '-'.join(demands_satisfied)

    # 4️⃣ Refuel stops y amounts
    refuel_stops = []
    refuel_amounts = []
    for s in model.S:
        qty = pyo.value(model.r[v, s])
        if qty > 0:
            refuel_stops.append(s)
            refuel_amounts.append(f"{qty:.1f}")
    refuel_stops_count = len(refuel_stops)
    refuel_amounts_string = '-'.join(refuel_amounts) if refuel_amounts else '0'

    # 5️⃣ Distancia total
    total_distance = 0
    for (i, j) in model.A:
        if pyo.value(model.x[v, i, j]) > 0.5:
            total_distance += distance(i, j)

    # 6️⃣ Otros datos
    vehicle_row = {
        'VehicleId': v,
        'LoadCap': vehicles.loc[vehicles['VehicleID'] == v, 'Capacity'].values[0],
        'FuelCap': Range[v],  # Usamos Range como la autonomía máxima
        'RouteSequence': route_string,
        'Municipalities': municipalities_count,
        'DemandSatisfied': demand_string,
        'InitLoad': sum(Demand[d] for d in municipalities_visited),
        'InitFuel': Range[v],
        'RefuelStops': refuel_stops_count,
        'RefuelAmounts': refuel_amounts_string,
        'Distance': f"{total_distance:.1f}",
        'Time': f"{total_distance / 50:.1f}",  # Ejemplo: suponemos velocidad promedio de 50 km/h
        'FuelCost': f"{sum(pyo.value(model.r[v, s]) * FuelPrice[s] for s in model.S):.0f}",
        'TotalCost': f"{pyo.value(model.TotalCost):.0f}"
    }
    verif_rows.append(vehicle_row)

# Crear DataFrame
df_verif = pd.DataFrame(verif_rows)

# Guardar CSV
df_verif.to_csv('verificacion_caso2.csv', index=False)

print("\n Archivo 'verificacion_caso2.csv' creado exitosamente.")



✅ Archivo 'verificacion_caso2.csv' creado exitosamente.
